In [1]:
#using K-Means clustering to build a product recommendation system from two different datasets
#one dataset has movies and the other has ratings made by users that have watched the movie; common field is movie_id


In [1]:
import zipfile
with zipfile.ZipFile('C:\Input\ml-latest-small.zip', 'r') as zip_ref:
    zip_ref.extractall('data')

In [2]:
# import the dataset
import pandas as pd
movies_df = pd.read_csv('data/ml-latest-small/movies.csv')
ratings_df = pd.read_csv('data/ml-latest-small/ratings.csv')

In [3]:
# Data Citation:
# F. Maxwell Harper and Joseph A. Konstan. 2015. The MovieLens Datasets: History and Context. ACM Transactions on 
# Interactive Intelligent Systems (TiiS) 5, 4: 19:1–19:19. <https://doi.org/10.1145/2827872>

! curl http://files.grouplens.org/datasets/movielens/ml-latest-small.zip -o ml-latest-small.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:--  0:00:01 --:--:--     0
  6  955k    6 59723    0     0  20897      0  0:00:46  0:00:02  0:00:44 20896
 33  955k   33  320k    0     0  87399      0  0:00:11  0:00:03  0:00:08 87406
 56  955k   56  540k    0     0   110k      0  0:00:08  0:00:04  0:00:04  110k
100  955k  100  955k    0     0   175k      0  0:00:05  0:00:05 --:--:--  195k


In [4]:
print('The dimensions of movies dataframe are:', movies_df.shape,'\nThe dimensions of ratings dataframe are:', ratings_df.shape)

The dimensions of movies dataframe are: (9742, 3) 
The dimensions of ratings dataframe are: (100836, 4)


In [5]:
# Take a look at movies_df
movies_df.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [6]:
# Take a look at ratings_df
ratings_df.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [7]:
# Movie ID to movie name mapping
movie_names = movies_df.set_index('movieId')['title'].to_dict()
n_users = len(ratings_df.userId.unique())
n_items = len(ratings_df.movieId.unique())
print("Number of unique users:", n_users)
print("Number of unique movies:", n_items)
print("The full rating matrix will have:", n_users*n_items, 'elements.')
print('----------')
print("Number of ratings:", len(ratings_df))
print("Therefore: ", len(ratings_df) / (n_users*n_items) * 100, '% of the matrix is filled.')
print("We have an incredibly sparse matrix to work with here.")
print("And... as you can imagine, as the number of users and products grow, the number of elements will increase by n*2")
print("You are going to need a lot of memory to work with global scale... storing a full matrix in memory would be a challenge.")
print("One advantage here is that matrix factorization can realize the rating matrix implicitly, thus we don't need all the data")

Number of unique users: 610
Number of unique movies: 9724
The full rating matrix will have: 5931640 elements.
----------
Number of ratings: 100836
Therefore:  1.6999683055613624 % of the matrix is filled.
We have an incredibly sparse matrix to work with here.
And... as you can imagine, as the number of users and products grow, the number of elements will increase by n*2
You are going to need a lot of memory to work with global scale... storing a full matrix in memory would be a challenge.
One advantage here is that matrix factorization can realize the rating matrix implicitly, thus we don't need all the data


In [8]:
import torch
import numpy as np
from torch.autograd import Variable
from tqdm import tqdm_notebook as tqdm

class MatrixFactorization(torch.nn.Module):
    def __init__(self, n_users, n_items, n_factors=20):
        super().__init__()
        # create user embeddings
        self.user_factors = torch.nn.Embedding(n_users, n_factors) # think of this as a lookup table for the input.
        # create item embeddings
        self.item_factors = torch.nn.Embedding(n_items, n_factors) # think of this as a lookup table for the input.
        self.user_factors.weight.data.uniform_(0, 0.05)
        self.item_factors.weight.data.uniform_(0, 0.05)
        
    def forward(self, data):
        # matrix multiplication
        users, items = data[:,0], data[:,1]
        return (self.user_factors(users)*self.item_factors(items)).sum(1)
    # def forward(self, user, item):
    # 	# matrix multiplication
    #     return (self.user_factors(user)*self.item_factors(item)).sum(1)
    
    def predict(self, user, item):
        return self.forward(user, item)

In [9]:
# Creating the dataloader (necessary for PyTorch)
from torch.utils.data.dataset import Dataset
from torch.utils.data import DataLoader # package that helps transform your data to machine learning readiness

# Note: This isn't 'good' practice, in a MLops sense but we'll roll with this since the data is already loaded in memory.
class Loader(Dataset):
    def __init__(self):
        self.ratings = ratings_df.copy()
        
        # Extract all user IDs and movie IDs
        users = ratings_df.userId.unique()
        movies = ratings_df.movieId.unique()
        
        #--- Producing new continuous IDs for users and movies ---
        
        # Unique values : index
        self.userid2idx = {o:i for i,o in enumerate(users)}
        self.movieid2idx = {o:i for i,o in enumerate(movies)}
        
        # Obtained continuous ID for users and movies
        self.idx2userid = {i:o for o,i in self.userid2idx.items()}
        self.idx2movieid = {i:o for o,i in self.movieid2idx.items()}
        
        # return the id from the indexed values as noted in the lambda function down below.
        self.ratings.movieId = ratings_df.movieId.apply(lambda x: self.movieid2idx[x])
        self.ratings.userId = ratings_df.userId.apply(lambda x: self.userid2idx[x])
        
        
        self.x = self.ratings.drop(['rating', 'timestamp'], axis=1).values
        self.y = self.ratings['rating'].values
        self.x, self.y = torch.tensor(self.x), torch.tensor(self.y) # Transforms the data to tensors (ready for torch models.)

    def __getitem__(self, index):
        return (self.x[index], self.y[index])

    def __len__(self):
        return len(self.ratings)

In [10]:
num_epochs = 128
cuda = torch.cuda.is_available()

print("Is running on GPU:", cuda)

model = MatrixFactorization(n_users, n_items, n_factors=8)
print(model)
for name, param in model.named_parameters():
    if param.requires_grad:
        print(name, param.data)
# GPU enable if you have a GPU...
if cuda:
    model = model.cuda()

# MSE loss
loss_fn = torch.nn.MSELoss()

# ADAM optimizier
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

# Train data
train_set = Loader()
train_loader = DataLoader(train_set, 128, shuffle=True)

Is running on GPU: False
MatrixFactorization(
  (user_factors): Embedding(610, 8)
  (item_factors): Embedding(9724, 8)
)
user_factors.weight tensor([[0.0413, 0.0199, 0.0413,  ..., 0.0373, 0.0431, 0.0192],
        [0.0226, 0.0485, 0.0135,  ..., 0.0077, 0.0352, 0.0247],
        [0.0054, 0.0462, 0.0134,  ..., 0.0053, 0.0185, 0.0271],
        ...,
        [0.0219, 0.0440, 0.0177,  ..., 0.0225, 0.0310, 0.0312],
        [0.0047, 0.0426, 0.0440,  ..., 0.0227, 0.0118, 0.0318],
        [0.0362, 0.0247, 0.0467,  ..., 0.0097, 0.0393, 0.0438]])
item_factors.weight tensor([[0.0367, 0.0446, 0.0325,  ..., 0.0021, 0.0281, 0.0372],
        [0.0235, 0.0152, 0.0220,  ..., 0.0439, 0.0271, 0.0098],
        [0.0273, 0.0371, 0.0190,  ..., 0.0238, 0.0040, 0.0465],
        ...,
        [0.0467, 0.0445, 0.0204,  ..., 0.0276, 0.0236, 0.0249],
        [0.0134, 0.0241, 0.0013,  ..., 0.0132, 0.0021, 0.0215],
        [0.0038, 0.0098, 0.0024,  ..., 0.0158, 0.0204, 0.0430]])


In [11]:
for it in tqdm(range(num_epochs)):
    losses = []
    for x, y in train_loader:
         if cuda:
            x, y = x.cuda(), y.cuda()
            optimizer.zero_grad()
            outputs = model(x)
            loss = loss_fn(outputs.squeeze(), y.type(torch.float32))
            losses.append(loss.item())
            loss.backward()
            optimizer.step()
            print("iter #{}".format(it), "Loss:", sum(losses) / len(losses))

<ipython-input-11-d512f2c473f0>:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for it in tqdm(range(num_epochs)):


  0%|          | 0/128 [00:00<?, ?it/s]

In [12]:
# By training the model, we will have tuned latent factors for movies and users.
c = 0
uw = 0
iw = 0 
for name, param in model.named_parameters():
    if param.requires_grad:
        print(name, param.data)
        if c == 0:
            uw = param.data
            c +=1
        else:
            iw = param.data
        #print('param_data', param_data)

user_factors.weight tensor([[0.0413, 0.0199, 0.0413,  ..., 0.0373, 0.0431, 0.0192],
        [0.0226, 0.0485, 0.0135,  ..., 0.0077, 0.0352, 0.0247],
        [0.0054, 0.0462, 0.0134,  ..., 0.0053, 0.0185, 0.0271],
        ...,
        [0.0219, 0.0440, 0.0177,  ..., 0.0225, 0.0310, 0.0312],
        [0.0047, 0.0426, 0.0440,  ..., 0.0227, 0.0118, 0.0318],
        [0.0362, 0.0247, 0.0467,  ..., 0.0097, 0.0393, 0.0438]])
item_factors.weight tensor([[0.0367, 0.0446, 0.0325,  ..., 0.0021, 0.0281, 0.0372],
        [0.0235, 0.0152, 0.0220,  ..., 0.0439, 0.0271, 0.0098],
        [0.0273, 0.0371, 0.0190,  ..., 0.0238, 0.0040, 0.0465],
        ...,
        [0.0467, 0.0445, 0.0204,  ..., 0.0276, 0.0236, 0.0249],
        [0.0134, 0.0241, 0.0013,  ..., 0.0132, 0.0021, 0.0215],
        [0.0038, 0.0098, 0.0024,  ..., 0.0158, 0.0204, 0.0430]])


In [13]:
trained_movie_embeddings = model.item_factors.weight.data.cpu().numpy()

In [14]:
len(trained_movie_embeddings) # unique movie factor weights

9724

In [15]:
from sklearn.cluster import KMeans
# Fit the clusters based on the movie weights
kmeans = KMeans(n_clusters=10, random_state=0).fit(trained_movie_embeddings)

In [ ]:
'''It can be seen here that the movies that are in the same cluster tend to have
similar genres. Also note that the algorithm is unfamiliar with the movie name
and only obtained the relationships by looking at the numbers representing how
users have responded to the movie selections.'''
for cluster in range(10):
    print("Cluster #{}".format(cluster))
    movs = []
    for movidx in np.where(kmeans.labels_ == cluster)[0]:
        movid = train_set.idx2movieid[movidx]
        rat_count = ratings_df.loc[ratings_df['movieId']==movid].count()[0]
        movs.append((movie_names[movid], rat_count))
        for mov in sorted(movs, key=lambda tup: tup[1], reverse=True)[:10]: 
           print("\t", mov[0])

Cluster #0
	 From Dusk Till Dawn (1996)
	 From Dusk Till Dawn (1996)
	 Canadian Bacon (1995)
	 Pulp Fiction (1994)
	 From Dusk Till Dawn (1996)
	 Canadian Bacon (1995)
	 Pulp Fiction (1994)
	 From Dusk Till Dawn (1996)
	 Canadian Bacon (1995)
	 Blown Away (1994)
	 Pulp Fiction (1994)
	 Full Metal Jacket (1987)
	 From Dusk Till Dawn (1996)
	 Canadian Bacon (1995)
	 Blown Away (1994)
	 Pulp Fiction (1994)
	 Full Metal Jacket (1987)
	 From Dusk Till Dawn (1996)
	 Canadian Bacon (1995)
	 Blown Away (1994)
	 Quiet Man, The (1952)
	 Pulp Fiction (1994)
	 Terminator, The (1984)
	 Full Metal Jacket (1987)
	 From Dusk Till Dawn (1996)
	 Canadian Bacon (1995)
	 Blown Away (1994)
	 Quiet Man, The (1952)
	 Pulp Fiction (1994)
	 Terminator, The (1984)
	 Full Metal Jacket (1987)
	 From Dusk Till Dawn (1996)
	 Pink Floyd: The Wall (1982)
	 Canadian Bacon (1995)
	 Blown Away (1994)
	 Quiet Man, The (1952)
	 Pulp Fiction (1994)
	 Terminator, The (1984)
	 Full Metal Jacket (1987)
	 Austin Powers: Intern